# Acid

In [1]:
!pip install fredapi

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import preprocessor as pre
import pandas as pd
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np
import matplotlib.pyplot as plt
from sktime.forecasting.naive import NaiveForecaster
import seaborn as sns

In [15]:
gas_df = pre.get_Fred_data('PNGASEUUSDM',2015,2021)
ammonia_df = pre.get_Fred_data('WPU0652013A',2015,2021)
wheat_df = pre.get_Fred_data('PWHEAMTUSDM',2015,2021)
elec_df = pre.clean_elec_csv('ELECTRICITY.csv',2015,2021)

price_evo_df = pre.clean_pred_price_evo_csv('Dataset_Future_Predicting_Price_Evolutions.csv',2016,2021)

dummy_df = pre.get_dummies_and_average_price(price_evo_df,'acid',\
                                         'RM01/0001','RM01/0004','RM01/0006','RM01/0007')

acid_df = pre.generate_features(1,12,dummy_df,\
                                Electricity=elec_df,\
                                PWHEAMTUSDM=wheat_df,\
                                WPU0652013A=ammonia_df,\
                                PNGASEUUSDM=gas_df)

# print(acid_df.info())

In [16]:
## train_test_split()
## Log transformation

# Create X, y
feature_list = acid_df.drop(['Time', 'Group Description', 'Year','Month','Average_price'],axis=1)
X = feature_list.values
y = acid_df['Average_price'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) # 30% of our data as the test set

# Log transformation and standardlisation
y_train_log = np.log(y_train)
y_test_log = np.log(y_test)

scaler_x = StandardScaler()
X_train_scaled = scaler_x.fit_transform(X_train)
X_test_scaled = scaler_x.transform(X_test)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train_log.reshape(-1,1))
y_test_scaled = scaler_y.transform(y_test_log.reshape(-1,1))

In [17]:
## Lasso regression - fit and transform train data set
## Cross validation and Hyperparameter tuning using RandomizedSearchCV

# Define the parameter grid
param_grid = {'alpha': np.linspace(0.0000001, 1, 3000)}

# Create a Lasso regression model
lasso = Lasso()

# Create RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=lasso, 
                                   param_distributions=param_grid, 
                                   n_iter=300, 
                                   cv=5, 
                                   random_state=42)

# Fit the data to perform a grid search
random_search.fit(X_train_scaled, y_train_scaled)

# Best alpha parameter
print("Best alpha parameter:", random_search.best_params_['alpha'])

# Best R-squared score
print("Best R-squared score:", round(random_search.best_score_, 3))

# Coefficients of the best Lasso model
assert random_search.n_features_in_ == len(feature_list.columns)

# print("Coefficients of the selected features in the best Lasso model:")
# for feature, coefficient in zip(feature_list.columns, random_search.best_estimator_.coef_):
#     print(f"{feature}: {round(coefficient,3)}")

Best alpha parameter: 1e-07
Best R-squared score: 0.917


In [18]:
## Lasso regression - transform test data set
# Get the best Lasso model from RandomizedSearchCV
best_lasso_model = random_search.best_estimator_

# Predict on the test data
y_pred_test = best_lasso_model.predict(X_test_scaled)

# Evaluate the model performance on the test data
test_score = best_lasso_model.score(X_test_scaled, y_test_scaled)
print("Best Model:", best_lasso_model)
print("Test Set R-squared score:", round(test_score, 3))



Best Model: Lasso(alpha=1e-07)
Test Set R-squared score: 0.912
